In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, DoubleType, ArrayType
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import uuid

# use in notebook only
%load_ext dotenv
%dotenv ../.env

# Load environment variables from the .env file in the parent directory
# dotenv_path = os.path.join(os.path.dirname(__file__), "..", ".env")
# load_dotenv(dotenv_path)

# PySpark Data Cleaning for PostgreSQL Stock Data

This PySpark script is designed for data cleaning and schema customization of stock data retrieved from a PostgreSQL database. The code covers the following steps:

1. **Setting Up Spark Session:**
   - Specifies the path to the PostgreSQL JDBC driver JAR file.
   - Creates a Spark session with specific configurations, including the JDBC driver.

2. **Reading Database Properties:**
   - Reads database connection properties from environment variables such as user, password, driver, and URL.

3. **Reading Data from PostgreSQL:**
   - Attempts to read stock data from the specified PostgreSQL table using the provided database properties.

4. **Defining Custom Schema:**
   - Defines a custom schema for the stock data, specifying data types for each column.

5. **Applying Custom Schema:**
   - Converts the data types of columns in the DataFrame to match the custom schema.

6. **Displaying the Result:**
   - Shows the cleaned stock data with the custom schema.

7. **Exception Handling:**
   - Catches and handles any exceptions that may occur during the data reading process, providing detailed error information.

Note: Make sure to replace placeholder values such as schema name, table name, and environment variables with actual values specific to your PostgreSQL setup.

The code aims to ensure that the stock data adheres to a defined schema for downstream analysis or processing.


In [2]:
# Specify the path to the PostgreSQL JDBC driver JAR file
postgres_jar_path = "./drivers/postgresql-42.6.0.jar"

# Create a Spark session with the PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName("StockDataCleaning") \
    .config("spark.executor.memory", "4g") \
    .config("spark.jars", postgres_jar_path) \
    .getOrCreate()

# Read database properties from environment variables
db_properties = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "driver": os.getenv("DB_DRIVER"),
    "url": os.getenv("DB_URL"),
}

# Schema and table name
schema_name = "public"  # Replace with your actual schema name
table_name = f'{schema_name}."Stocks"'

# Attempt to read data from PostgreSQL
try:
    stockData = spark.read.jdbc(url=db_properties["url"],
                                table=table_name,
                                properties=db_properties)

    # Define your custom schema
    custom_schema = StructType([
        StructField("transaction_id", StringType(), True),
        StructField("stock_id", StringType(), True),
        StructField("ticker_symbol", StringType(), True),
        StructField("date", DateType(), True),
        StructField("low", FloatType(), True),
        StructField("open", FloatType(), True),
        StructField("high", FloatType(), True),
        StructField("volume", IntegerType(), True),
        StructField("close", FloatType(), True)
    ])

    # Apply the custom schema to the DataFrame
    stockData = stockData \
        .withColumn("transaction_id", stockData["transaction_id"].cast(StringType())) \
        .withColumn("stock_id", stockData["stock_id"].cast(StringType())) \
        .withColumn("ticker_symbol", stockData["ticker_symbol"].cast(StringType())) \
        .withColumn("date", stockData["date"].cast(DateType())) \
        .withColumn("low", stockData["low"].cast(FloatType())) \
        .withColumn("open", stockData["open"].cast(FloatType())) \
        .withColumn("high", stockData["high"].cast(FloatType())) \
        .withColumn("volume", stockData["volume"].cast(IntegerType())) \
        .withColumn("close", stockData["close"].cast(FloatType()))

    # Show the DataFrame with the custom schema
    stockData.show()

except Exception as e:
    print("Error reading data from PostgreSQL:")
    print(e)
    # Print the full stack trace for debugging
    import traceback
    traceback.print_exc()


23/12/03 17:22:38 WARN Utils: Your hostname, michaelwoan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.0.140 instead (on interface enp0s3)
23/12/03 17:22:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/03 17:22:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/03 17:22:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+--------------------+--------------------+-------------+----------+--------+--------+--------+---------+--------+
|      transaction_id|            stock_id|ticker_symbol|      date|     low|    open|    high|   volume|   close|
+--------------------+--------------------+-------------+----------+--------+--------+--------+---------+--------+
|f6026c92-1a47-403...|9169dbe9-0d59-486...|         GOOG|2004-08-19|2.390042|2.490664|2.591785|897427216|2.499133|
|e50595f9-83c8-4e6...|9169dbe9-0d59-486...|         GOOG|2004-08-20|2.503118| 2.51582|2.716817|458857488|2.697639|
|e7665eab-e706-45d...|9169dbe9-0d59-486...|         GOOG|2004-08-23| 2.71607|2.758411|2.826406|366857939|2.724787|
|f456c078-0976-4f0...|9169dbe9-0d59-486...|         GOOG|2004-08-24|2.579581|2.770615|2.779581|306396159| 2.61196|
|a424e353-c6b8-4e3...|9169dbe9-0d59-486...|         GOOG|2004-08-25|2.587302|2.614201|2.689918|184645512|2.640104|
|dc77880b-700f-47f...|9169dbe9-0d59-486...|         GOOG|2004-08-26|2.606729|2.6

## Data Preprocessing: Selecting Latest Stock Data

This cell performs data preprocessing on the stock data. It first converts the 'date' column to a date type if it's not already. Then, it creates a window specification for each ticker symbol, ordering the data by date in descending order.

A row number is added to the DataFrame based on the window specification, and rows with a row number equal to 1 (representing the latest date) for each group are filtered. The unnecessary 'row_number' column is dropped, and only the essential columns (ticker symbol, date, low, open, high, volume, and close) are selected in the final result.

The processed result is displayed using the `show()` function.


In [3]:
# Convert the 'date' column to a date type if it's not already
stockData = stockData.withColumn("date", F.to_date(stockData["date"]))

# Create a window specification for each group, ordered by the 'date' column in descending order
windowSpec = Window().partitionBy("ticker_symbol").orderBy(F.desc("date"))

# Add a row number to the DataFrame based on the window specification
rankedData = stockData.withColumn("row_number", F.row_number().over(windowSpec))

# Filter the rows with row number equal to 1 (latest date) for each group
latestData = rankedData.filter("row_number = 1").drop("row_number")

# Select only the necessary columns
result = latestData.select("ticker_symbol", "date", "low", "open", "high", "volume", "close")

# Show the result
result.show()

+-------------+----------+-------+------+-------+---------+-------+
|ticker_symbol|      date|    low|  open|   high|   volume|  close|
+-------------+----------+-------+------+-------+---------+-------+
|         AAPL|2023-11-21| 189.74|191.41|  191.5| 36857230| 190.64|
|         ADBE|2023-12-01| 603.18|609.24|  615.2|  2490000| 612.47|
|         AMZN|2023-12-01| 145.55| 146.0| 147.25| 39924600| 147.03|
|          AXP|2023-12-01| 170.31|170.67| 174.34|  3412000| 173.76|
|         BABA|2023-12-01|  72.12| 73.12|  74.14| 27946500|  73.99|
|           FL|2023-12-01|  27.03| 27.03|  29.72|  6112300|  29.71|
|         GOOG|2023-11-21|137.705| 135.5|138.965| 14530014|138.435|
|         INTC|2023-12-01|  42.86| 44.36|  44.36| 43922500|  43.74|
|         MSFT|2023-12-01| 371.31|376.76| 378.16| 33020400| 374.51|
|          NKE|2023-12-01| 109.75|110.33| 113.55|  8049900| 113.48|
|         NVDA|2023-12-01| 461.87|465.25|  472.0| 36880900| 467.65|
|         PYPL|2023-12-01|  57.45| 57.63|  59.81

## Data Quality Check: Missing Values

This cell performs a check for missing values in the stock data. It calculates the sum of null values for each column and prints the results.

In [4]:
# Missing Values
missing_values = stockData.select([func.sum(func.col(c).isNull().cast("int")).alias(c + '_missing') for c in stockData.columns]).collect()

print("Missing Values:")
for row in missing_values[0].asDict():
    print(f"{row}: {missing_values[0][row]}")


Missing Values:
transaction_id_missing: 0
stock_id_missing: 0
ticker_symbol_missing: 0
date_missing: 0
low_missing: 0
open_missing: 0
high_missing: 0
volume_missing: 0
close_missing: 0


## Data Cleaning: Handling Missing Values and Deduplication

This cell addresses data quality by filling missing values in the 'volume' column with 0. Additionally, it removes duplicate rows based on the 'date' and 'close' columns, ensuring data integrity.

In [5]:
# Fill missing values in the 'volume' column with 0
stockData = stockData.na.fill(0, subset=['volume'])

# Drop duplicate rows based on 'date' and 'close' columns
cleanedStockData = stockData.dropDuplicates(['date', 'close'])

# Check for duplicate values in the 'date' column again
duplicate_rows = cleanedStockData.groupBy('date', 'close').count().filter('count > 1')

# Show the duplicate dates and close prices, if any
if duplicate_rows.count() > 0:
    print("Duplicate dates and close prices found after deduplication:")
    duplicate_rows.show()
else:
    print("No duplicate dates and close prices found.")

cleanedStockData.orderBy(func.desc("date")).show()


No duplicate dates and close prices found.


+--------------------+--------------------+-------------+----------+------+------+------+---------+------+
|      transaction_id|            stock_id|ticker_symbol|      date|   low|  open|  high|   volume| close|
+--------------------+--------------------+-------------+----------+------+------+------+---------+------+
|462e276f-d094-40e...|daaa9ed4-4caa-46f...|          UPS|2023-12-01|151.02|151.99|154.97|  3945000|154.91|
|88101aec-f12b-409...|e390ddf0-6791-474...|         AMZN|2023-12-01|145.55| 146.0|147.25| 39924600|147.03|
|f603e7ae-39b2-4a4...|060bfc23-efbd-4e9...|         BABA|2023-12-01| 72.12| 73.12| 74.14| 27946500| 73.99|
|2c049a81-df3b-4ed...|7de656af-463b-494...|          WMT|2023-12-01|152.63|155.38|155.61| 13085300|154.34|
|10850edd-c86e-40f...|b15241c9-1e87-417...|           FL|2023-12-01| 27.03| 27.03| 29.72|  6112300| 29.71|
|8c60b38d-9ba8-4a4...|15a05405-6476-426...|         INTC|2023-12-01| 42.86| 44.36| 44.36| 43922500| 43.74|
|605802cd-ed4c-4fa...|057b007d-d7f5-4

# Drop Views to Update tables otherwise will get errors 

In [6]:
import sys
sys.path.append('..')  # Add the parent directory to the Python path

# Create a database connection
from database import create_connection, execute_sql

connection = create_connection()

drop_views = ['DROP MATERIALIZED VIEW IF EXISTS stock_technical_view;']

execute_sql(connection, drop_views)

SQL statements executed successfully.


## Stock Data Moving Averages Analysis

This PySpark script calculates Simple Moving Averages (SMA) and Exponential Moving Averages (EMA) for different periods on stock data. The analysis includes importing libraries, defining functions, setting parameters, and displaying the results.

- **Moving Averages:**
  - Simple Moving Averages (SMA) for periods: 5, 20, 50, 200.
  - Exponential Moving Averages (EMA) with corresponding alpha values.

- **Data Manipulation:**
  - Utilizes PySpark functions and windows for efficient data processing.

- **Result Display:**
  - Presents the DataFrame with date, close price, SMAs, and EMAs in descending order.


In [7]:
# Define a UDF to generate UUIDs
@F.udf(StringType())
def generate_uuid():
    return str(uuid.uuid4())

round_to_decimal = 2

# Generate a UUID for each row
# cleanedStockData = cleanedStockData.withColumn("cal_id", F.lit(str(uuid.uuid4())))
cleanedStockData = cleanedStockData.withColumn("cal_id", generate_uuid())

def calculate_ema(data, alpha):
    ema = data[0]
    for i in range(1, len(data)):
        ema = alpha * data[i] + (1 - alpha) * ema
    return ema

calculate_ema_udf = F.udf(lambda data, alpha: float(calculate_ema(data, alpha)), FloatType())

periods = [5, 20, 50, 200]
alpha_values = [2 / (p + 1) for p in periods]

partition_cols = ["stock_id", "ticker_symbol"]

windows = [Window().partitionBy(partition_cols).orderBy(F.desc("date")).rowsBetween(0, p - 1) for p in periods]

# Calculate simple moving averages
for p in periods:
    cleanedStockData = cleanedStockData.withColumn(f"{p}_days_sma", F.round(F.avg("close").over(windows[periods.index(p)]), 2))

# Calculate exponential moving averages using UDF
for p, alpha in zip(periods, alpha_values):
    cleanedStockData = cleanedStockData.withColumn(f"{p}_days_ema", F.round(calculate_ema_udf(F.collect_list("close").over(windows[periods.index(p)]), F.lit(alpha)), round_to_decimal))

# Show the result
moving_averages_data = cleanedStockData.select(['cal_id','transaction_id',"stock_id", "ticker_symbol",'date'] + [f"{p}_days_sma" for p in periods] + [f"{p}_days_ema" for p in periods]).orderBy(F.desc("date"))
moving_averages_data.show()

# Schema and table name
table_name = '"MovingAverages"'

# Write the DataFrame to the database table
moving_averages_data.write.jdbc(os.getenv("DB_URL"), table_name, mode="overwrite", properties=db_properties)


+--------------------+--------------------+--------------------+-------------+----------+----------+-----------+-----------+------------+----------+-----------+-----------+------------+
|              cal_id|      transaction_id|            stock_id|ticker_symbol|      date|5_days_sma|20_days_sma|50_days_sma|200_days_sma|5_days_ema|20_days_ema|50_days_ema|200_days_ema|
+--------------------+--------------------+--------------------+-------------+----------+----------+-----------+-----------+------------+----------+-----------+-----------+------------+
|33e3590a-bd51-436...|2c049a81-df3b-4ed...|7de656af-463b-494...|          WMT|2023-12-01|     156.3|     160.48|     160.76|      154.46|     156.5|     161.51|     160.01|      151.18|
|bac69a54-ae7a-405...|8ae69ba2-ac88-48b...|669c2fc6-f1bd-45b...|         ADBE|2023-12-01|    616.69|     600.78|     558.49|       468.9|    617.73|     594.34|     549.84|       448.8|
|e66b6bd3-a961-480...|a220e2ee-95bb-466...|26a37152-781c-400...|      

## Bollinger Bands Calculation Explanation

This Jupiter Notebook cell performs the computation of Bollinger Bands on stock data for volatility analysis. The breakdown includes critical steps and considerations:

- **Decimal Rounding:**
  - All numerical values are rounded to two decimal places for consistency and readability.

- **Bollinger Bands Periods:**
  - The Bollinger Bands are computed for four distinct periods: 5, 20, 50, and 200 days, providing insights into short-term and long-term volatility.

- **Partitioning for Accuracy:**
  - The data is partitioned by "stock_id" and "ticker_symbol" to ensure accurate calculations for individual stocks. This is crucial for meaningful stock market analysis.

- **Reuse of Exponential Moving Averages (EMAs):**
  - Existing EMA values, previously calculated, are reused in the Bollinger Bands computation. This approach optimizes computational efficiency and maintains consistency with prior analyses.

- **Upper and Lower Band Calculation:**
  - The upper and lower bands are determined by adding and subtracting twice the standard deviation of closing prices from the corresponding EMAs. This methodology aligns with the standard Bollinger Bands formula.

- **Result Presentation:**
  - The final DataFrame includes the date, close price, upper bands, and lower bands for each specified period, providing a comprehensive view of the stock's volatility.

This code enhances the dataset with Bollinger Bands, aiding in the identification of potential market trends and volatility patterns.

In [8]:
# Define the Bollinger Bands periods
bollinger_periods = [5, 20, 50, 200]

partition_cols = ["stock_id", "ticker_symbol"]

# Define the windows for Bollinger Bands
windows = [Window().partitionBy(partition_cols).orderBy(F.desc("date")).rowsBetween(0, p - 1) for p in bollinger_periods]

# Reuse the existing EMA values for Bollinger Bands
for p in bollinger_periods:
    upper_band_col = F.col(f"{p}_days_ema") + (2 * F.stddev("close").over(windows[bollinger_periods.index(p)]))
    lower_band_col = F.col(f"{p}_days_ema") - (2 * F.stddev("close").over(windows[bollinger_periods.index(p)]))

    cleanedStockData = cleanedStockData.withColumn(f"{p}_upper_band", F.round(upper_band_col, round_to_decimal))
    cleanedStockData = cleanedStockData.withColumn(f"{p}_lower_band", F.round(lower_band_col, round_to_decimal))

# Show the result
selected_columns = ['cal_id','transaction_id',"stock_id", "ticker_symbol",'date'] + [f"{p}_upper_band" for p in bollinger_periods] + [f"{p}_lower_band" for p in bollinger_periods]
boilling_bands_data = cleanedStockData.select(selected_columns).orderBy(F.desc("date"))
boilling_bands_data.show()

# Schema and table name
table_name = '"BoillingerBands"'

# Write the DataFrame to the database table
boilling_bands_data.write.jdbc(os.getenv("DB_URL"), table_name, mode="overwrite", properties=db_properties)


23/12/03 17:23:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+-------------+----------+------------+-------------+-------------+--------------+------------+-------------+-------------+--------------+
|              cal_id|      transaction_id|            stock_id|ticker_symbol|      date|5_upper_band|20_upper_band|50_upper_band|200_upper_band|5_lower_band|20_lower_band|50_lower_band|200_lower_band|
+--------------------+--------------------+--------------------+-------------+----------+------------+-------------+-------------+--------------+------------+-------------+-------------+--------------+
|2eec9a16-1a5c-4c7...|88101aec-f12b-409...|e390ddf0-6791-474...|         AMZN|2023-12-01|      148.37|       148.78|       150.29|         151.1|      145.77|       138.18|       116.67|         85.66|
|6cc67eaf-e98a-4c7...|8ae69ba2-ac88-48b...|669c2fc6-f1bd-45b...|         ADBE|2023-12-01|      627.79|       630.26|       628.73|         626.1|      607.67|       558.42|       470.95|      

## Relative Strength Index (RSI) Calculation Explanation

This code cell calculates the Relative Strength Index (RSI) for a given stock dataset. The RSI is a momentum oscillator that measures the speed and change of price movements. The breakdown includes critical steps and considerations:

- **Price Changes:**
  - The code calculates the daily price changes by subtracting the previous day's closing price from the current day's closing price.

- **Gains and Losses:**
  - Gains and losses are determined based on whether the price change is positive or negative.

- **Exponential Moving Averages (EMAs):**
  - Exponential Moving Averages (EMAs) are calculated for both gains and losses over the specified RSI period. EMAs give more weight to recent price changes, providing a more responsive indicator.

- **Handling NULL and Zero Values:**
  - NULL values for average gains are handled by replacing them with zero. Additionally, zero values for average losses are replaced with zero to avoid division errors.

- **Relative Strength (RS) Calculation:**
  - The Relative Strength (RS) is calculated as the ratio of average gains to average losses.

- **RSI Calculation:**
  - The RSI is calculated using the standard formula: \(100 - \frac{100}{1 + RS}\). This value is then rounded to two decimal places.

- **List of RSI Periods:**
  - The code calculates RSI for multiple periods, including 14, 20, 50, and 200 days, providing insights into short-term and long-term momentum.

- **Result Presentation:**
  - The final DataFrame includes columns for the ticker symbol, date, closing price, and RSI for each specified period, offering a comprehensive view of the stock's momentum trends.

- **Data Partitioning:**
  - The data is partitioned by "stock_id" and "ticker_symbol" to ensure accurate calculations for individual stocks. This is crucial for meaningful stock market analysis.

- **Decimal Rounding:**
  - All numerical values, including the calculated RSI, are rounded to two decimal places for consistency and readability.

This code enhances the dataset with RSI values, contributing to the analysis of potential overbought or oversold market conditions.


In [9]:
def calculate_rsi(data, n):
    # Calculate price changes
    price_diff = F.col("close") - F.lag("close", 1).over(Window().partitionBy("stock_id", "ticker_symbol").orderBy("date"))
    
    # Separate gains and losses
    gains = F.when(price_diff > 0, price_diff).otherwise(0)
    losses = F.when(price_diff < 0, -price_diff).otherwise(0)
    
    # Calculate average gains and losses over n periods from the latest day backward
    avg_gains = F.avg(gains).over(Window().partitionBy("stock_id", "ticker_symbol").orderBy(F.desc("date")).rowsBetween(0, n-1))
    avg_losses = F.avg(losses).over(Window().partitionBy("stock_id", "ticker_symbol").orderBy(F.desc("date")).rowsBetween(0, n-1))
    
    # Handle NULL values for average gains
    avg_gains = F.coalesce(avg_gains, F.lit(0))

    # Handle 0 values for average losses
    avg_losses = F.when(avg_losses.isNull() | (avg_losses == 0), 0).otherwise(avg_losses)

    # Calculate RSI
    rs = F.when((avg_losses == 0) & (avg_gains != 0), F.lit(avg_gains)) \
      .when((avg_losses != 0) & (avg_gains == 0), F.lit(0)) \
      .otherwise(F.when(avg_losses == 0, F.lit(float('inf'))) \
                  .otherwise(avg_gains / avg_losses))
    
    # Calculate RSI and round to 2 decimal places
    rsi = 100 - (100 / (1 + rs))
    
    return F.round(rsi, round_to_decimal)

# List of RSI periods
rsi_periods = [14, 20, 50, 200]

# Calculate and add RSI columns to the DataFrame for each period
for n in rsi_periods:
    column_name = f"{n}_days_rsi"
    cleanedStockData = cleanedStockData.withColumn(column_name, calculate_rsi(cleanedStockData, n))

# Show the result
result_columns = ['cal_id','transaction_id',"stock_id", "ticker_symbol",'date'] + [f"{n}_days_rsi" for n in rsi_periods]
relative_indexes_data = cleanedStockData.select(result_columns).orderBy(F.desc("date"))
relative_indexes_data.show()

# Schema and table name
table_name = '"RelativeIndexes"'

relative_indexes_data.write.jdbc(os.getenv("DB_URL"), table_name, mode="overwrite", properties=db_properties)

# Stop the Spark session when you're done
spark.stop()


+--------------------+--------------------+--------------------+-------------+----------+-----------+-----------+-----------+------------+
|              cal_id|      transaction_id|            stock_id|ticker_symbol|      date|14_days_rsi|20_days_rsi|50_days_rsi|200_days_rsi|
+--------------------+--------------------+--------------------+-------------+----------+-----------+-----------+-----------+------------+
|d242ca41-b348-45f...|88101aec-f12b-409...|e390ddf0-6791-474...|         AMZN|2023-12-01|      59.09|      65.56|      58.99|       56.28|
|e3ba0f19-ac2c-409...|8ae69ba2-ac88-48b...|669c2fc6-f1bd-45b...|         ADBE|2023-12-01|      60.03|      70.73|      63.89|       58.53|
|bfdef06a-c8de-403...|f603e7ae-39b2-4a4...|060bfc23-efbd-4e9...|         BABA|2023-12-01|      29.65|      32.04|      41.76|       45.51|
|9e1fba12-f27d-4c1...|b90c0fa1-dc9a-4f2...|a5295793-3807-4db...|         PYPL|2023-12-01|      66.14|      61.84|      51.11|       45.84|
|7dca369d-1dcb-458...|80459

# Update tables to original structures after overwriten process

## Alter tables to approriate data types 

In [10]:
alter_statements = [
    # Altered data types for MovingAverages table
    """
    ALTER TABLE "MovingAverages"
    ALTER COLUMN "cal_id" TYPE UUID USING "transaction_id"::UUID,
    ALTER COLUMN "transaction_id" TYPE UUID USING "transaction_id"::UUID,
    ALTER COLUMN "stock_id" TYPE UUID USING "stock_id"::UUID,
    ALTER COLUMN "ticker_symbol" TYPE VARCHAR,
    ALTER COLUMN "date" TYPE DATE,
    ALTER COLUMN "5_days_sma" TYPE FLOAT,
    ALTER COLUMN "20_days_sma" TYPE FLOAT,
    ALTER COLUMN "50_days_sma" TYPE FLOAT,
    ALTER COLUMN "200_days_sma" TYPE FLOAT,
    ALTER COLUMN "5_days_ema" TYPE FLOAT,
    ALTER COLUMN "20_days_ema" TYPE FLOAT,
    ALTER COLUMN "50_days_ema" TYPE FLOAT,
    ALTER COLUMN "200_days_ema" TYPE FLOAT;
    """,
    # Altered data types for BoillingerBands table
    """
    ALTER TABLE "BoillingerBands"
    ALTER COLUMN "cal_id" TYPE UUID USING "transaction_id"::UUID,
    ALTER COLUMN "transaction_id" TYPE UUID USING "transaction_id"::UUID,
    ALTER COLUMN "stock_id" TYPE UUID USING "stock_id"::UUID,
    ALTER COLUMN "ticker_symbol" TYPE VARCHAR,
    ALTER COLUMN "date" TYPE DATE,
    ALTER COLUMN "5_upper_band" TYPE FLOAT,
    ALTER COLUMN "20_upper_band" TYPE FLOAT,
    ALTER COLUMN "50_upper_band" TYPE FLOAT,
    ALTER COLUMN "200_upper_band" TYPE FLOAT,
    ALTER COLUMN "5_lower_band" TYPE FLOAT,
    ALTER COLUMN "20_lower_band" TYPE FLOAT,
    ALTER COLUMN "50_lower_band" TYPE FLOAT,
    ALTER COLUMN "200_lower_band" TYPE FLOAT;
    """,

    # Altered data types for RelativeIndexes table
    """
    ALTER TABLE "RelativeIndexes"
    ALTER COLUMN "cal_id" TYPE UUID USING "transaction_id"::UUID,
    ALTER COLUMN "transaction_id" TYPE UUID USING "transaction_id"::UUID,
    ALTER COLUMN "stock_id" TYPE UUID USING "stock_id"::UUID,
    ALTER COLUMN "ticker_symbol" TYPE VARCHAR,
    ALTER COLUMN "date" TYPE DATE,
    ALTER COLUMN "14_days_rsi" TYPE FLOAT,
    ALTER COLUMN "20_days_rsi" TYPE FLOAT,
    ALTER COLUMN "50_days_rsi" TYPE FLOAT,
    ALTER COLUMN "200_days_rsi" TYPE FLOAT;
    """
]

# Execute SQL statements
execute_sql(connection, alter_statements)

SQL statements executed successfully.


## Adding key constrains

In [11]:
alter_key_constrains_statements = [
    # MovingAverages
    'ALTER TABLE "MovingAverages" ADD CONSTRAINT "pk_MovingAverages_cal_id" PRIMARY KEY ("cal_id");',
    'ALTER TABLE "MovingAverages" ADD CONSTRAINT "fk_MovingAverages_transaction_id" FOREIGN KEY ("transaction_id") REFERENCES "Stocks"("transaction_id") ON DELETE CASCADE;',
    'ALTER TABLE "MovingAverages" ADD CONSTRAINT "fk_MovingAverages_stock_id_ticker_symbol" FOREIGN KEY ("stock_id", "ticker_symbol") REFERENCES "CompanyInformation"("stock_id", "ticker_symbol");',

    # BoillingerBands
    'ALTER TABLE "BoillingerBands" ADD CONSTRAINT "pk_BoillingerBands_cal_id" PRIMARY KEY ("cal_id");',
    'ALTER TABLE "BoillingerBands" ADD CONSTRAINT "fk_BoillingerBands_transaction_id" FOREIGN KEY ("transaction_id") REFERENCES "Stocks"("transaction_id") ON DELETE CASCADE;',
    'ALTER TABLE "BoillingerBands" ADD CONSTRAINT "fk_BoillingerBands_stock_id_ticker_symbol" FOREIGN KEY ("stock_id", "ticker_symbol") REFERENCES "CompanyInformation"("stock_id", "ticker_symbol");',

    # RelativeIndexes
    'ALTER TABLE "RelativeIndexes" ADD CONSTRAINT "pk_RelativeIndexes_cal_id" PRIMARY KEY ("cal_id");',
    'ALTER TABLE "RelativeIndexes" ADD CONSTRAINT "fk_RelativeIndexes_transaction_id" FOREIGN KEY ("transaction_id") REFERENCES "Stocks"("transaction_id") ON DELETE CASCADE;',
    'ALTER TABLE "RelativeIndexes" ADD CONSTRAINT "fk_RelativeIndexes_stock_id_ticker_symbol" FOREIGN KEY ("stock_id", "ticker_symbol") REFERENCES "CompanyInformation"("stock_id", "ticker_symbol");'
]

# Execute SQL statements
execute_sql(connection, alter_key_constrains_statements)

SQL statements executed successfully.


## Create Indexes

In [12]:
# List of SQL statements for indexes
index_sql_statements = [
    'CREATE INDEX IF NOT EXISTS idx_ma_stock_id_ticker_symbol_date ON "MovingAverages"("stock_id", "ticker_symbol", "date");',
    'CREATE INDEX IF NOT EXISTS idx_ma_date ON "MovingAverages"("date");',
    'CREATE INDEX IF NOT EXISTS idx_bb_stock_id_ticker_symbol_date ON "BoillingerBands"("stock_id", "ticker_symbol", "date");',
    'CREATE INDEX IF NOT EXISTS idx_bb_date ON "BoillingerBands"("date");',
    'CREATE INDEX IF NOT EXISTS idx_ri_stock_id_ticker_symbol_date ON "RelativeIndexes"("stock_id", "ticker_symbol", "date");',
    'CREATE INDEX IF NOT EXISTS idx_ri_date ON "RelativeIndexes"("date");',
]

# Execute SQL statements
execute_sql(connection, index_sql_statements)


SQL statements executed successfully.


In [13]:
# Close the database connection
connection.close()